In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam

# Settings
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 15  # Start with 15, can increase to 25-30
TRAIN_DIR = '/kaggle/input/brain-tumor-mri-dataset/Training'
VAL_DIR = '/kaggle/input/brain-tumor-mri-dataset/Testing'
NUM_CLASSES = 4

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Transfer Learning Model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze base

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.4)(x)  # helps reduce overfitting
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile Model
model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train Top Layers
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

# Fine-tune Some Base Layers
# Unfreeze last few layers for fine-tuning
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune
history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE
)

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 177s 960ms/step - accuracy: 0.4573 - loss: 1.3402 - val_accuracy: 0.7361 - val_loss: 0.6486
Epoch 2/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 199s 945ms/step - accuracy: 0.7620 - loss: 0.6310 - val_accuracy: 0.7643 - val_loss: 0.5714
Epoch 3/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 169s 944ms/step - accuracy: 0.8056 - loss: 0.5140 - val_accuracy: 0.7948 - val_loss: 0.5115
Epoch 4/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/step - accuracy: 0.8266 - loss: 0.4610 - val_accuracy: 0.7887 - val_loss: 0.5401
Epoch 5/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.8377 - loss: 0.4330 - val_accuracy: 0.8230 - val_loss: 0.4680
Epoch 6/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 169s 941ms/step - accuracy: 0.8462 - loss: 0.4110 - val_accuracy: 0.8299 - val_loss: 0.4478
Epoch 7/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 170s 947ms/step - accuracy: 0.8408 - loss